In [ ]:
2+2

In [ ]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import  pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter # to create chunks
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [ ]:
from langchain_community.llms import CTransformers

In [ ]:
PINECONE_API_KEY = "97e0cd8a-16bb-433c-83ec-2291a1141456"
PINECONE_API_ENV = "gcp-starter"

### Extract Data From PDF

In [ ]:
def load_pdf(data):
   loader =  DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
   
   documents = loader.load()
   return documents

In [ ]:
extracted_data = load_pdf("data/")

# 

### Create Chunks

In [ ]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [ ]:
text_chunks = text_split(extracted_data)
print(len(text_chunks))

In [ ]:
len(text_chunks[2].page_content)

### download embedding model

In [ ]:
def download_hugging_face_embedding():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [ ]:
embedding = download_hugging_face_embedding()

In [ ]:
embedding

In [ ]:
query_result = embedding.embed_query("Hello World")
print(query_result)
print(len(query_result))

In [ ]:
query_result = embedding.embed_query(text_chunks[2].page_content)

In [ ]:
from pinecone import Pinecone
# pinecone.init(api_key=PINECONE_API_ENV,
#               environment = PINECONE_API_ENV)

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medical-chat-bot"

In [ ]:
# docsearch = Pinecone.from_texts([t.page_content for t in text_chunks], 
#                                 embedding, index_name = index_name
#                                 )
# DEPRICATED

In [ ]:
len(text_chunks)

In [ ]:
# docsearch = Pinecone.from_texts([t.page_content for t in text_chunks], 
#                                 embedding, index_name = "medical-chat-bot"
#                                 )

In [ ]:
# RUN THIS CODE AGAIN WHEN PUSHING DATA TO NEW INDEX
index = pc.Index(index_name)  
for i, t in zip(range(len(text_chunks)), text_chunks):
   query_result = embedding.embed_query(t.page_content)
   index.upsert(
   vectors=[
        {
            "id": str(i),  # Convert i to a string
            "values": query_result, 
            "metadata": {"text":str(text_chunks[i].page_content)} # meta data as dic
        }
    ],
    namespace="real" 
)

index.describe_index_stats() 

In [ ]:
from langchain.vectorstores import Pinecone as PineconeStore

In [ ]:
question = "What are Alergies?"
query_results = index.query(
    namespace="real",
    vector=embedding.embed_query(question),
    top_k=3,
    include_values=True
)

In [ ]:
query_results

In [ ]:
len(query_results.to_dict())

In [ ]:
type(query_results)

In [ ]:
query_results.matches[0]

In [ ]:
# from langchain_community.llms import CTransformers

In [ ]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [ ]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [ ]:
llm = None
path_to_model = r'C:\Users\Naruto\Desktop\generative_ai\generative_ai_material\project\Medical_Chat_Bot\model\llama-2-7b-chat.ggmlv3.q4_0.bin'
llm = CTransformers(
    model=path_to_model,
    model_type='llama',
    config={'max_new_tokens':552,
            'temperature':0.8}
    )

In [ ]:
# from langchain_core.output_parsers import StrOutputParser
# from langchain_core.runnables import RunnablePassthrough

In [ ]:
# def format_docs(docs):
#     return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
# def format_docs(docs):
#     return "\n\n".join(doc.page_content for doc in docs)

# vectordb = PineconeStore(index, embedding, text_key="text")
# qa_chain = (
#     {
#         "context": vectordb.as_retriever() | format_docs,
#         "question": RunnablePassthrough(),
#     }
#     | PROMPT
#     | llm
#     | StrOutputParser()
# )

# qa_chain.invoke("What are Alergies?")

In [ ]:
vectordb = PineconeStore(index, embedding, text_key="text")
qa=RetrievalQA.from_llm(
    llm=llm, 
    chain_type="stuff", 
    retriever=vectordb.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [ ]:
question = "What are Alergies?"

In [ ]:
result=qa.invoke({"query": question})

In [ ]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])

## hit and trial

In [ ]:
vectordb.as_retriever().invoke("What are Alergies?")

In [ ]:
<form id="apiKeyForm" class="input-group mb-3">
								<input type="text" id="api_key" name="api_key" placeholder="Enter your API key..." class="form-control" required/>
                                <div class="input-group-append">
									<button type="submit" id="send" class="input-group-text send_btn"><i class="fas fa-location-arrow"></i></button>
								</div>
							</form>